In [ ]:
import numpy as np
import glob
import os
from tqdm.notebook import tqdm
import scipy.linalg as la
from numpy.linalg import matrix_power
import math
import scipy.stats as sp
from statsmodels.stats.multitest import multipletests
import matplotlib as mlt
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import spearmanr

cmap = sns.color_palette("colorblind")

%run -i colorblind.py

In [ ]:
cmap_new = rainbow_colors(12)

fig, ax = plt.subplots(1,1,figsize=(3,1))
for i, color in enumerate(cmap_new): 
    ax.scatter(i, 1, color=color)

In [ ]:
def graph_metrics(adj_mat): 

    def eigenvector_centrality_und(adj_mat): 
        # Compute eigen vector centrality from undirected graph
        if np.isnan(np.min(adj_mat)): # Skip subjects with nan edges
            centrality = np.nan 
        else: 
            w, v = la.eig(adj_mat)
            idx = np.argmax(w)
            centrality = np.abs(v[:,idx])
        return centrality

    def clustering_coef_wu(adj_mat): 
        # Compute clustering coefficient from weighted undirected graph
        K = np.sum(adj_mat != 0, axis=1)            	
        cyc3 = np.diag(matrix_power(adj_mat**(1/3),3));           
        if len(np.where(cyc3 == 0)[0]) == 0: 
            CC = cyc3/(K*(K-1))
        else:
            K[np.where(cyc3 == 0)[0]] = np.nan
        return CC

    def strengths_und(adj_mat):
        str_und = np.sum(adj_mat, axis = 1)
        return str_und
    
    centrality = eigenvector_centrality_und(adj_mat)
    CC = clustering_coef_wu(adj_mat)
    str_und = strengths_und(adj_mat)
    
    return centrality, CC, str_und


def create_frequency_axis(f_min=2, f_max= 4, scale_freq ='log'):
    
    if scale_freq == 'log':
        f_min_log = math.floor(np.log10(f_min))
        f_max_log = math.ceil(np.log10(f_max))
        mb = np.logspace(f_min_log, f_max_log, num=40) # Morlet bank 
        freq_vals = mb[(2.1 < mb) & (mb < 75)] # Frequency of interest
        
    
    elif scale_freq == 'lin':
        freq_vals = np.linspace(f_min, f_max, f_max-1)

    freq_labels = [('%.2f'%x) for x in freq_vals]
    
    return freq_vals, freq_labels

def extract_fei_new(files):
    
    data, name_subjs, data_masked = list(), list(), list()
   
    for ifile, file in enumerate(tqdm(files)):
        
        name_subjs.append(file.split('/')[9][:6])
        
        tmp = np.load(file, allow_pickle=True)

        data.append(tmp[1]) 
        data_masked.append(np.nanmean(tmp[1,...],axis=0)) # tmp[1] == rimuovo fEI se 0.5 < cent < 1 non è rispettato
    
    fEI_ch = np.array(data)
    fEI_masked = np.array(data_masked)

    return name_subjs, fEI_ch, fEI_masked

def extract_wpli_new(files):
    
    name_subj, wpli_obs = list(), list()

    for ifile, file in enumerate(tqdm(files)):
        
        data = np.load(file, allow_pickle=True)
        
        _, nCh, _, nF = data.shape

        for d in data:
            for fIdx in range(nF):
                np.fill_diagonal(d[...,fIdx], 0)
                
        _, _, obs, _, _, _= data

        obs_wpli = np.zeros((nCh,nCh, nF))
         
        for iF in range(nF):
            
            obs_wpli[...,iF] = np.abs(obs[...,iF])

        wpli_obs.append(obs_wpli)
    
    wpli_obs = np.array(wpli_obs)

    return wpli_obs



In [ ]:
bids_root = '.../RBD_root/'
path_out = '.../Results/'
lut_filename = 'lut.xlsx'

# Set frequency limits
f_min = 2
f_max = 90
scale_freq='log'

#### Import wPLI

In [ ]:
wpliR1 = extract_wpli_new(sorted(glob.glob(os.path.join(wpli_root + 'connectivity/run-01/','sub*'))))
wpliR2 = extract_wpli_new(sorted(glob.glob(os.path.join(wpli_root + 'connectivity/run-02/','sub*'))))

In [ ]:
# Import LOOK UP TABLE - RBD 
lut_rbd = pd.read_excel(bids_root + lut_filename).set_index('ID')

# Extract information about conveterted and not converted patients
name_lateRBD_r1 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 1) ].index.to_list()
name_lateRBD_r1 = [i.split('_', 1)[0] for i in name_lateRBD_r1]
name_lateRBD_r2 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 1)  ].index.to_list()
name_lateRBD_r2 = [i.split('_', 1)[0] for i in name_lateRBD_r2]

name_earlyRBD_r1 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 2) ].index.to_list()
name_earlyRBD_r1 = [i.split('_', 1)[0] for i in name_earlyRBD_r1]
name_earlyRBD_r2 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 2) ].index.to_list()
name_earlyRBD_r2 = [i.split('_', 1)[0] for i in name_earlyRBD_r2]

name_sRBD_r1 = lut_rbd.loc[(lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 0)].index.to_list()
name_sRBD_r1 = [i.split('_', 1)[0] for i in name_sRBD_r1]
name_sRBD_r2 = lut_rbd.loc[(lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 0)].index.to_list()
name_sRBD_r2 = [i.split('_', 1)[0] for i in name_sRBD_r2]

#### Compute local graph metrics

In [ ]:
nsR1, nCh, _, nF = wpliR1.shape
nsR2, _, _, _ = wpliR2.shape

centralityR1 = np.zeros((nsR1, nCh, nF))
ccR1 = np.zeros_like(centralityR1)
strR1 = np.zeros_like(centralityR1)

centralityR2 = np.zeros((nsR2, nCh, nF))
ccR2 = np.zeros_like(centralityR2)
strR2 = np.zeros_like(centralityR2)

for iS in range(nsR1):
    for iF in range(nF):
        centralityR1[iS, :, iF], ccR1[iS, :, iF], strR1[iS,:,iF] = graph_metrics(wpliR1[iS,:,:,iF])

        
for iS in range(nsR2):
    for iF in range(nF):
        centralityR2[iS, :, iF], ccR2[iS, :, iF], strR2[iS,:,iF] = graph_metrics(wpliR2[iS,:,:,iF])

#### Import fEI

In [ ]:
name_subj_r1, _, fei_r1 = extract_fei_new(sorted(glob.glob(os.path.join(fei_root + 'fei/run-01/','*fei06*'))))
name_subj_r2, _, fei_r2 = extract_fei_new(sorted(glob.glob(os.path.join(fei_root + 'fei/run-02/','*fei06*'))))

In [ ]:
idx_r1_lateRBD = [name_subj_r1.index(x) for x in name_lateRBD_r1 if x in name_subj_r1]
idx_r2_lateRBD = [name_subj_r2.index(x) for x in name_lateRBD_r2 if x in name_subj_r2]

idx_r1_earlyRBD = [name_subj_r1.index(x) for x in name_earlyRBD_r2 if x in name_subj_r1]
idx_r2_earlyRBD = [name_subj_r2.index(x) for x in name_earlyRBD_r2 if x in name_subj_r2]

idx_r1_sRBD = [name_subj_r1.index(x) for x in name_sRBD_r1 if x in name_subj_r1]
idx_r2_sRBD = [name_subj_r2.index(x) for x in name_sRBD_r2 if x in name_subj_r2]

In [ ]:
freq_vals, _ = create_frequency_axis(f_min=f_min, f_max=f_max, scale_freq =scale_freq)

In [ ]:
freq_vals = freq_vals[:30]
fei_r1 = fei_r1[:,:30]
fei_r2 = fei_r2[:,:30]
fei = np.concatenate((fei_r1,fei_r2))
centralityR1 = np.mean(centralityR1, axis=1)
centralityR2 = np.mean(centralityR2, axis=1)
centrality = np.concatenate((centralityR1,centralityR2))
ccR1 = np.nanmean(ccR1[:,:30], axis=1)
ccR2 = np.nanmean(ccR2[:,:30], axis=1)
cc = np.concatenate((ccR1,ccR2))
strR1 = np.nanmean(strR1[:,:30], axis=1)
strR2 = np.nanmean(strR2[:,:30], axis=1)
strenght = np.concatenate((strR1,strR2))

#### Spearman's correlation and BH correction

In [ ]:
# Correlazione
freq_range = (np.linspace(0,9,10, dtype='int'),  # delta - theta
             np.linspace(10,21,12, dtype='int'), # alpha - beta
             np.linspace(22,29,8, dtype='int'))  # gamma

rho = np.zeros((len(freq_range),))
p = np.zeros_like(rho)

for i, iF in enumerate(freq_range): 
    
    rho[i], p[i] = spearmanr(fei[:,iF].mean(axis=1), updrs, nan_policy='omit')
    
# Benjiamini - Hochberg's correction
_, p_corrected, _, _ = multipletests(p, method='fdr_bh')

In [ ]:
def spearman_corr(data1, data2):
    _, nF = data1.shape
    rho = np.zeros((nF,))
    p = np.zeros_like(rho)
    
    # Spearman's correlation
    for iF in range(nF):
        
        # Check nan 
        idx_dat_d1 = np.argwhere(np.isnan(data1[:,iF]))
        data1_tmp = np.delete(data1[:,iF], idx_dat_d1)
        data2_tmp = np.delete(data2[:,iF], idx_dat_d1)
        
        idx_dat_d2 = np.argwhere(np.isnan(data2_tmp))
        data1_tmp = np.delete(data1_tmp, idx_dat_d2)
        data2_tmp = np.delete(data2_tmp, idx_dat_d2)
        
        rho[iF], p[iF] =  sp.spearmanr(data1_tmp, data2_tmp)
    
    # Benjiamini - Hochberg's correction
    _, p_corrected, _, _ = multipletests(p, method='fdr_bh')
    
    return rho, p, p_corrected

rho_cent, p_cent, p_cent_corrected = spearman_corr(centrality, fei)
rho_cc, p_cc, p_cc_corrected = spearman_corr(cc, fei)
rho_str, p_str, p_str_corrected = spearman_corr(strenght, fei)

In [ ]:
cc_fei_f, p_fei_f = spearmanr(fei, nan_policy='omit', alternative='two-sided')
cc_str_f, p_str_f = spearmanr(strenght, nan_policy='omit', alternative='two-sided')
cc_cc_f, p_cc_f = spearmanr(cc, nan_policy='omit', alternative='two-sided')
cc_centrality_f, p_centrality_f = spearmanr(centrality, nan_policy='omit', alternative='two-sided')

In [ ]:
m_size=6
l_size=8
a_size=6

fig, axs = plt.subplot_mosaic([['a', 'a', 'b', 'e'], ['a', 'a', 'c', 'f'], ['a', 'a', 'd', 'g']], layout='constrained', figsize=(17.6/2.54,7/2.54))

axs['a'].semilogx(freq_vals, rho_cc, color=cmap[1], label='Clustering coefficient');
axs['a'].semilogx(freq_vals, rho_str, color=cmap[2], label='Strength');
        
axs['a'].semilogx(freq_vals, rho_cent, color=cmap[7], label='Eigen vector centrality');

for iF, nF in enumerate(freq_vals):
    if p_cc[iF] < 0.05: 
        axs['a'].scatter(nF, 0.7, marker='o', 
                    facecolors='none', edgecolors=cmap[1], s=m_size)
    if p_cc_corrected[iF] < 0.05: 
        axs['a'].scatter(nF, 0.7, facecolors=cmap[1],
                    edgecolors=cmap[1], marker='o', s=m_size)
        
for iF, nF in enumerate(freq_vals):
    if p_str[iF] < 0.05: 
        axs['a'].scatter(nF, 0.725, marker='o', 
                    facecolors='none', edgecolors=cmap[2], s=m_size)
    if p_str_corrected[iF] < 0.05: 
        axs['a'].scatter(nF, 0.725, facecolors=cmap[2],
                    edgecolors=cmap[2], marker='o', s=m_size)

for iF, nF in enumerate(freq_vals):
    if p_cent[iF] < 0.05: 
        axs['a'].scatter(nF, 0.75, marker='o', 
                    facecolors='none', edgecolors=cmap[7], s=m_size)
    if p_cent_corrected[iF] < 0.05: 
        axs['a'].scatter(nF, 0.75, facecolors=cmap[7],
                    edgecolors=cmap[7], marker='o', s=m_size)
        
axs['a'].set_ylim([-.6, .8])
axs['a'].set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])
axs['a'].set_xlabel('Frequencies [Hz]', fontsize=10);
axs['a'].set_ylabel('CC' + '\n' + 'nodal wPLI vs fEI', fontsize=l_size);
axs['a'].legend(loc='lower left', fontsize=a_size, frameon=False);

# First


df = pd.DataFrame({'fEI (5-7)Hz':fei[:,7:11].mean(axis=1),
                   'wPLI (5-7)Hz':cc[:,7:11].mean(axis=1)})

sns.regplot(x='fEI (5-7)Hz', y='wPLI (5-7)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7], 'linewidth':.5}, ax=axs['b'])


# non-converters
axs['b'].scatter(fei_r1[idx_r1_sRBD,7:11].mean(axis=1), 
            ccR1[idx_r1_sRBD,7:11].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['b'].scatter(fei_r2[idx_r2_sRBD,7:11].mean(axis=1), 
            ccR2[idx_r2_sRBD, 7:11].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['b'].scatter(fei_r1[idx_r1_lateRBD,7:11].mean(axis=1), 
            ccR1[idx_r1_lateRBD, 7:11].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['b'].scatter(fei_r2[idx_r2_lateRBD,7:11].mean(axis=1), 
            ccR2[idx_r2_lateRBD, 7:11].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['b'].scatter(fei_r1[idx_r1_earlyRBD,7:11].mean(axis=1), 
            ccR1[idx_r1_earlyRBD, 7:11].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['b'].scatter(fei_r2[idx_r2_earlyRBD,7:11].mean(axis=1), 
            ccR2[idx_r2_earlyRBD, 7:11].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)


axs['b'].set_ylabel('Clustering' + '\n' +  'Coefficient' + '\n' + '(5-7Hz)', fontsize=l_size)
axs['b'].set_xlabel('fEI (5-7Hz)', fontsize=l_size)


#Second

df = pd.DataFrame({'fEI (8-13)Hz':fei[:,11:16].mean(axis=1),
                   'wPLI (8-13)Hz':cc[:,11:16].mean(axis=1)})

sns.regplot(x='fEI (8-13)Hz', y='wPLI (8-13)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7],  'linewidth':.5}, ax=axs['c'])


# non-converters
axs['c'].scatter(fei_r1[idx_r1_sRBD,11:16].mean(axis=1), 
            ccR1[idx_r1_sRBD,11:16].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['c'].scatter(fei_r2[idx_r2_sRBD,11:16].mean(axis=1), 
            ccR2[idx_r2_sRBD, 11:16].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['c'].scatter(fei_r1[idx_r1_lateRBD,11:16].mean(axis=1), 
            ccR1[idx_r1_lateRBD, 11:16].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['c'].scatter(fei_r2[idx_r2_lateRBD,11:16].mean(axis=1), 
            ccR2[idx_r2_lateRBD, 11:16].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['c'].scatter(fei_r1[idx_r1_earlyRBD,11:16].mean(axis=1), 
            ccR1[idx_r1_earlyRBD, 11:16].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['c'].scatter(fei_r2[idx_r2_earlyRBD,11:16].mean(axis=1), 
            ccR2[idx_r2_earlyRBD, 11:16].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)

axs['c'].set_ylabel('Clustering' + '\n' +  'Coefficient' + '\n' + '(8-13Hz)', fontsize=l_size)
axs['c'].set_xlabel('fEI (8-13Hz)', fontsize=l_size)


#Third
df = pd.DataFrame({'fEI (27-38)Hz':fei[:,21:25].mean(axis=1),
                   'wPLI (27-38)Hz':cc[:,21:25].mean(axis=1)})

sns.regplot(x='fEI (27-38)Hz', y='wPLI (27-38)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7],  'linewidth':.5}, ax=axs['d'])


# non-converters
axs['d'].scatter(fei_r1[idx_r1_sRBD,21:25].mean(axis=1), 
            ccR1[idx_r1_sRBD,21:25].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['d'].scatter(fei_r2[idx_r2_sRBD,21:25].mean(axis=1), 
            ccR2[idx_r2_sRBD, 21:25].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['d'].scatter(fei_r1[idx_r1_lateRBD,21:25].mean(axis=1), 
            ccR1[idx_r1_lateRBD, 21:25].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['d'].scatter(fei_r2[idx_r2_lateRBD,21:25].mean(axis=1), 
            ccR2[idx_r2_lateRBD, 21:25].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['d'].scatter(fei_r1[idx_r1_earlyRBD,21:25].mean(axis=1), 
            ccR1[idx_r1_earlyRBD, 21:25].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['d'].scatter(fei_r2[idx_r2_earlyRBD,21:25].mean(axis=1), 
            ccR2[idx_r2_earlyRBD, 21:25].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)

axs['d'].set_ylabel('Clustering' + '\n' +  'Coefficient' + '\n' + '(27-38Hz)', fontsize=l_size)
axs['d'].set_xlabel('fEI (27-38Hz)', fontsize=l_size)


# First
df = pd.DataFrame({'fEI (5-7)Hz':fei[:,7:11].mean(axis=1),
                   'wPLI (5-7)Hz':strenght[:,7:11].mean(axis=1)})

sns.regplot(x='fEI (5-7)Hz', y='wPLI (5-7)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7], 'linewidth':.5}, ax=axs['e'])


# non-converters
axs['e'].scatter(fei_r1[idx_r1_sRBD,7:11].mean(axis=1), 
            strR1[idx_r1_sRBD,7:11].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['e'].scatter(fei_r2[idx_r2_sRBD,7:11].mean(axis=1), 
            ccR2[idx_r2_sRBD, 7:11].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['e'].scatter(fei_r1[idx_r1_lateRBD,7:11].mean(axis=1), 
            strR1[idx_r1_lateRBD, 7:11].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['e'].scatter(fei_r2[idx_r2_lateRBD,7:11].mean(axis=1), 
            strR2[idx_r2_lateRBD, 7:11].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['e'].scatter(fei_r1[idx_r1_earlyRBD,7:11].mean(axis=1), 
            strR1[idx_r1_earlyRBD, 7:11].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['e'].scatter(fei_r2[idx_r2_earlyRBD,7:11].mean(axis=1), 
            strR2[idx_r2_earlyRBD, 7:11].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)

axs['e'].set_ylabel('Strength' + '\n' + '(5-7Hz)', fontsize=l_size)
axs['e'].set_xlabel('fEI (5-7Hz)', fontsize=l_size)

#Second
df = pd.DataFrame({'fEI (8-13)Hz':fei[:,11:17].mean(axis=1),
                   'wPLI (8-13)Hz':strenght[:,11:17].mean(axis=1)})

sns.regplot(x='fEI (8-13)Hz', y='wPLI (8-13)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7], 'linewidth':.5}, ax=axs['f'])

# non-converters
axs['f'].scatter(fei_r1[idx_r1_sRBD,11:17].mean(axis=1), 
            strR1[idx_r1_sRBD,11:17].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['f'].scatter(fei_r2[idx_r2_sRBD,11:17].mean(axis=1), 
            ccR2[idx_r2_sRBD, 11:17].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['f'].scatter(fei_r1[idx_r1_lateRBD,11:17].mean(axis=1), 
            strR1[idx_r1_lateRBD,11:17].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['f'].scatter(fei_r2[idx_r2_lateRBD,11:17].mean(axis=1), 
            strR2[idx_r2_lateRBD, 11:17].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['f'].scatter(fei_r1[idx_r1_earlyRBD,11:17].mean(axis=1), 
            strR1[idx_r1_earlyRBD, 11:17].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['f'].scatter(fei_r2[idx_r2_earlyRBD,11:17].mean(axis=1), 
            strR2[idx_r2_earlyRBD, 11:17].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)

axs['f'].set_ylabel('Strength' + '\n' + '(8-15Hz)', fontsize=l_size)
axs['f'].set_xlabel('fEI (8-15Hz)', fontsize=l_size)



#Third
df = pd.DataFrame({'fEI (27-38)Hz':fei[:,22:25].mean(axis=1),
                   'wPLI (27-38)Hz':strenght[:,22:25].mean(axis=1)})

sns.regplot(x='fEI (27-38)Hz', y='wPLI (27-38)Hz', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7], 'linewidth':.5}, ax=axs['g'])

# non-converters
axs['g'].scatter(fei_r1[idx_r1_sRBD,22:25].mean(axis=1), 
            strR1[idx_r1_sRBD,22:25].mean(axis=1), marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
axs['g'].scatter(fei_r2[idx_r2_sRBD,22:25].mean(axis=1), 
            ccR2[idx_r2_sRBD, 22:25].mean(axis=1), marker='o', 
               color=cmap_new[5], s=m_size) 

# late converters
axs['g'].scatter(fei_r1[idx_r1_lateRBD,22:25].mean(axis=1), 
            strR1[idx_r1_lateRBD,22:25].mean(axis=1), marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
axs['g'].scatter(fei_r2[idx_r2_lateRBD,22:25].mean(axis=1), 
            strR2[idx_r2_lateRBD, 22:25].mean(axis=1), marker='^', 
            color=cmap_new[9], s=m_size) 

# early converters
axs['g'].scatter(fei_r1[idx_r1_earlyRBD,22:25].mean(axis=1), 
            strR1[idx_r1_earlyRBD,22:25].mean(axis=1), marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
axs['g'].scatter(fei_r2[idx_r2_earlyRBD,22:25].mean(axis=1), 
            strR2[idx_r2_earlyRBD, 22:25].mean(axis=1), marker='X', color=cmap_new[11], 
            s=m_size)

axs['g'].set_ylabel('Strength' + '\n' + '(30-38Hz)', fontsize=l_size)
axs['g'].set_xlabel('fEI (30-38Hz)', fontsize=l_size)



legend = axs['c'].legend(loc='upper left', fontsize=6);
legend.get_frame().set_facecolor('none')
legend.get_frame().set_edgecolor('none')
axs['c'].set_facecolor(color='None')

axs['a'].tick_params(labelsize=a_size); 
axs['b'].tick_params(labelsize=a_size); 
axs['c'].tick_params(labelsize=a_size); 
axs['d'].tick_params(labelsize=a_size); 
axs['e'].tick_params(labelsize=a_size); 
axs['f'].tick_params(labelsize=a_size); 
axs['g'].tick_params(labelsize=a_size); 

axs['a'].spines[['top', 'right']].set_visible(False)
axs['b'].spines[['top', 'right']].set_visible(False)
axs['c'].spines[['top', 'right']].set_visible(False)
axs['d'].spines[['top', 'right']].set_visible(False)
axs['e'].spines[['top', 'right']].set_visible(False)
axs['f'].spines[['top', 'right']].set_visible(False)
axs['g'].spines[['top', 'right']].set_visible(False)

plt.savefig(path_out + 'Crit_F6.svg', dpi=600, bbox_inches = "tight")
plt.savefig(path_out + 'Crit_F6.png', dpi=300, bbox_inches = "tight")

In [ ]:
import pandas as pd 
df = pd.DataFrame()
df['Frequencies [Hz]'] = freq_vals
df['$\rho$ cent'] = rho_cent
df['$\it{p}$ cent'] =  p_cent
df['$\it{p}$ cent corrected'] =  p_cent_corrected

df['$\rho$ cc'] = rho_cc
df['$\it{p}$ cc'] =  p_cc
df['$\it{p}$ cc corrected'] =  p_cc_corrected

df['$\rho$ str'] = rho_str
df['$\it{p}$ str'] =  p_str
df['$\it{p}$ str corrected'] =  p_str_corrected

df.set_index('Frequencies [Hz]').to_excel('SupplementaryTables_F7.xlsx', sheet_name='Figure7')